In [60]:
import pandas as pd

def add_space(x):
    if (' ' not in x)&(len(x)<=5):
        return ' ' + x + ' '
    else:
        return x

def product_name_postprocess(x):
    x = x.replace('-',' ')
    x = x.strip()
    x = add_space(x)
    return x

df = pd.read_excel('../data/寶典.v4.20211001.xlsx',engine='openpyxl')
df = df.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df['品名'] = df['品名'].apply(lambda x:product_name_postprocess(x))
df['RIGID'].fillna('not value',inplace=True)
df['EXTN'].fillna('not value',inplace=True)
品名2部門 = dict(zip(df['品名'],df['公司事業部門']))
品名2代號 = dict(zip(df['品名'],df['公司代號']))
NAME = 'EVA 7350M'

In [61]:
品名2部門[NAME]

'台塑聚烯部'

In [62]:
品名2代號[NAME]

'18'

In [63]:
train_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv')
train_df['Y_label'] = train_df['Y_label'].apply(lambda x:x.strip()) #針對SPEC去除左右空白
train_df[(train_df['Y_label']==NAME)]

,Unnamed: 0,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2


In [64]:
# 根據品名從訓練資料搜索EXPNO,然後把EXPNO代入保典裡找公司部門
def find_department(x):
    try:
        return df.loc[df['公司代號']==train_df.loc[train_df.Y_label==x,'EXPNO'].dropna().value_counts().sort_values().index[-1],'公司事業部門'].value_counts().sort_values().index[-1]
    except:
        return 'not from_pretrained'

In [65]:
find_department(NAME)

'not from_pretrained'

In [66]:
df[df['品名']==NAME]

,RIGID,EXTN,公司代號,公司事業部門,品名
594,李佩珍,6445,18,台塑聚烯部,EVA 7350M


In [67]:
train_df[train_df['Y_label']==NAME]

,Unnamed: 0,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
